## Step 1: Import Required Libraries

In [ ]:
import json
import sys
from pathlib import Path
from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

print("Libraries imported successfully!")

## Step 2: Load Episode Logs

Adjust the `log_path` below to point to your episode log file. It's typically located at `engine/logs/episode.log`.

In [ ]:
# Path to the episode log file
log_path = Path("/Users/ebo/Code/solitaire/engine/logs/episode.log")

if not log_path.exists():
    print(f"⚠️  Log file not found at {log_path}")
    print("Generate logs with: ./gradlew test --tests ai.games.results.AStarPlayerResultsTest -Dlog.episodes=true")
else:
    print(f"✅ Found log file: {log_path}")
    print(f"   File size: {log_path.stat().st_size:,} bytes")

## Step 3: Parse Episode Data

Extract metrics from each game step:
- **Foundation cards**: How many cards are in the foundation piles
- **Face-down cards**: How many cards remain hidden in the tableau
- **Progress**: Foundation progress (0-1) and revelation progress (0-1)

In [ ]:
# Data structure to hold game information
games = defaultdict(lambda: {
    'steps': [],
    'won': None,
    'initial_facedown': None,
    'num_moves': 0
})

# Parse the log file
try:
    with open(log_path, 'r') as f:
        line_count = 0
        for line in f:
            line_count += 1
            if line.startswith('EPISODE_STEP '):
                data = json.loads(line[len('EPISODE_STEP '):])
                game_idx = data['game_index']
                
                # Extract metrics
                foundation_cards = sum(len(suit) for suit in data['foundation'])
                current_facedown = sum(data['tableau_face_down'])
                
                # Initialize facedown count on first step
                if games[game_idx]['initial_facedown'] is None:
                    games[game_idx]['initial_facedown'] = current_facedown
                
                initial_facedown = games[game_idx]['initial_facedown']
                games[game_idx]['num_moves'] += 1
                games[game_idx]['won'] = data.get('won', False)
                
                # Compute progress metrics
                foundation_progress = foundation_cards / 52.0
                revelation_progress = 0.0
                if initial_facedown > 0:
                    revealed = initial_facedown - current_facedown
                    revelation_progress = revealed / initial_facedown
                
                games[game_idx]['steps'].append({
                    'move': data.get('move_index', 0),
                    'foundation': foundation_cards,
                    'facedown': current_facedown,
                    'foundation_progress': foundation_progress,
                    'revelation_progress': revelation_progress
                })
    
    print(f"✅ Parsed {line_count:,} lines")
    print(f"✅ Found {len(games)} games")
except FileNotFoundError:
    print(f"❌ Could not open {log_path}")
except json.JSONDecodeError as e:
    print(f"❌ JSON parsing error: {e}")

## Step 4: Summary Statistics

In [ ]:
if len(games) > 0:
    # Compute statistics
    wins = sum(1 for g in games.values() if g['won'])
    losses = len(games) - wins
    win_rate = (wins / len(games)) * 100 if len(games) > 0 else 0
    
    moves = [g['num_moves'] for g in games.values()]
    avg_moves = np.mean(moves)
    
    print("=" * 50)
    print("GAME SUMMARY")
    print("=" * 50)
    print(f"Total Games:     {len(games)}")
    print(f"Games Won:       {wins}")
    print(f"Games Lost:      {losses}")
    print(f"Win Rate:        {win_rate:.1f}%")
    print(f"\nAvg Moves/Game:  {avg_moves:.1f}")
    print(f"Min Moves:       {min(moves)}")
    print(f"Max Moves:       {max(moves)}")
    print("=" * 50)
else:
    print("No games found in log file")

## Step 5: Visualize Game Progression

Let's visualize how foundation cards and face-down cards change over the course of a few games.

In [ ]:
if len(games) > 0:
    # Pick a few games to visualize
    sample_games = sorted(games.keys())[:3]  # First 3 games
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Foundation cards over time
    for game_idx in sample_games:
        steps = games[game_idx]['steps']
        moves = [s['move'] for s in steps]
        foundation = [s['foundation'] for s in steps]
        won = "✓ Won" if games[game_idx]['won'] else "✗ Lost"
        axes[0].plot(moves, foundation, marker='o', label=f"Game {game_idx} {won}", alpha=0.7)
    
    axes[0].set_xlabel("Move Number")
    axes[0].set_ylabel("Cards in Foundation")
    axes[0].set_title("Foundation Progress")
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_ylim([0, 52])
    
    # Plot 2: Face-down cards over time
    for game_idx in sample_games:
        steps = games[game_idx]['steps']
        moves = [s['move'] for s in steps]
        facedown = [s['facedown'] for s in steps]
        won = "✓ Won" if games[game_idx]['won'] else "✗ Lost"
        axes[1].plot(moves, facedown, marker='o', label=f"Game {game_idx} {won}", alpha=0.7)
    
    axes[1].set_xlabel("Move Number")
    axes[1].set_ylabel("Face-Down Cards Remaining")
    axes[1].set_title("Card Revelation Progress")
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    print(f"Plotted {len(sample_games)} sample games")
else:
    print("No data to visualize")

## Step 6: Compute Heuristic Values

The neural network is trained to predict a value (win probability) based on board state. 
A simple heuristic combines foundation progress and revelation progress:

```
value = (foundation_progress × 0.4) + (revelation_progress × 0.6)
```

Let's see how this evolves during a typical game.

In [ ]:
if len(games) > 0:
    # Pick one game and compute heuristic value at each step
    game_to_analyze = sorted(games.keys())[0]
    steps = games[game_to_analyze]['steps']
    
    heuristic_values = []
    for step in steps:
        value = (step['foundation_progress'] * 0.4) + (step['revelation_progress'] * 0.6)
        heuristic_values.append(value)
    
    # Plot heuristic progression
    plt.figure(figsize=(10, 5))
    moves = [s['move'] for s in steps]
    plt.plot(moves, heuristic_values, marker='o', linewidth=2, markersize=4)
    plt.fill_between(moves, heuristic_values, alpha=0.3)
    
    plt.xlabel("Move Number")
    plt.ylabel("Heuristic Value (Win Probability Estimate)")
    plt.title(f"Game {game_to_analyze} - Heuristic Value Over Time")
    plt.grid(True, alpha=0.3)
    plt.ylim([0, 1])
    
    won = "✓ Won" if games[game_to_analyze]['won'] else "✗ Lost"
    plt.text(0.5, 0.95, f"Result: {won}", transform=plt.gca().transAxes, 
             ha='center', va='top', fontsize=12, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nHeuristic Value Statistics for Game {game_to_analyze}:")
    print(f"  Initial: {heuristic_values[0]:.3f}")
    print(f"  Final:   {heuristic_values[-1]:.3f}")
    print(f"  Average: {np.mean(heuristic_values):.3f}")
    print(f"  Max:     {max(heuristic_values):.3f}")

## Summary

You've now learned how to:
1. **Load episode logs** from the Java engine
2. **Parse JSON data** and extract game metrics
3. **Compute statistics** like win rates and move counts
4. **Visualize game progression** to understand how foundation and revelation progress evolve
5. **Calculate heuristic values** that estimate game quality

This is the foundation for understanding what the neural network needs to learn to predict.

### Next Steps
- Modify the heuristic weights to see how they affect predictions
- Load logs from different AI players and compare their strategies
- Analyze which game states are most predictive of wins